This notebook will pull other data that we plan to use for the CS 109 final projects. Planning on bringing in education, demographic, and economic data to help us predict crime

In [1]:
import pandas as pd
import urllib
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
"""
Function
---------------
split_MSA

This method takes in a dataframe with MSA and splits into a city_key (largest city)
and state_key. This will help facilitate MSA merging

Returns dataframe with these two additional features
"""
def split_MSA(df):
    df['MSA'] = df['MSA'].str.replace('Metro Area', '')
    # Need to manually fix how this MSA is written
    df.loc[df['MSA'].str.contains("Texarkana"), "MSA"] = "Texarkana, AR-TX"

    #Grab Everything before comma
    df['city_key'] = df['MSA'].str.split(",").str[0]
    # Then grab everything before first hyphen if it has it
    df['city_key'] = df['city_key'].str.split("-").str[0].str.strip()
    # State will be everying after comma 
    df['state_key']=df['MSA'].str.split(",").str[1].str.strip()
    return(df)

"""
Function
--------
append_df

This function appends two dataframes

Parameters:
    input - dataframe to be appended
    output - dataframe to be appended onto
    
Returns a single dataframe 
"""
def append_df(input,output):
    if output.empty:
        output=input.copy()
    else:
        output=pd.concat([output,input])
        output.reset_index(drop='Index',inplace=True)
    return(output)

'''
Function
-----------
var_thresh

This function takes in a dataframe and keeps only thos varaibles that have a pct
non-missing that is above that threshold
'''
def var_thresh(df, thresh=0.65):
    return(df.loc[:, pd.notnull(df).sum() > len (df) *thresh])

'''
Function
---------
slim_df

This function takes in a list of variables to keep
on the the given df. It keep the variables + geography
then renames to MSA and drops the first row of variable descriptions
'''
def slim_df(df, var_list):
    var_list.append('GEO.display-label')
    df = df.loc[:, var_list]
    # Get rid of Micro Areas
    df = df.loc[~df['GEO.display-label'].str.contains("Micro Area"), :]
    
    df = df.rename(index=str, columns={'GEO.display-label': 'MSA'})
    df['MSA'] = df["MSA"].astype(str)
    # Drop first row of var descriptions
    df = df.loc[df.MSA != "Geography", :]
    # Split MSA into city-state key
    return(split_MSA(df))

In [3]:
'''
Function
---------
match_crime

This function will take in a dataframe and make changes to MSA
in order to match crime data
'''
def match_crime(df):
    df.loc[df['MSA'].str.contains('Crestview'),'city_key']='Crestview'
    df.loc[df['MSA'].str.contains('Sarasota'),'city_key']='North Port'
    df.loc[df['MSA'].str.contains('Louisville'),'city_key']='Louisville'
    df.loc[df['MSA'].str.contains('Santa Maria'),'city_key']='Santa Maria'
    df.loc[df['MSA'].str.contains('Weirton'),'city_key']='Weirton'
    df.loc[df['MSA'].str.contains('San Germán'),'city_key']='San German'
    df.loc[df['MSA'].str.contains('Mayagüez'),'city_key']='Mayaguez'
    df.loc[df['MSA'].str.contains('Honolulu'),'city_key']='Urban Honolulu'

    #State
    df.loc[df['MSA'].str.contains('Worcester'),'state_key']='MA-CT'
    df.loc[df['MSA'].str.contains('Myrtle Beach'),'state_key']='SC-NC'
    df.loc[df['MSA'].str.contains('Salisbury'),'state_key']='MD-DE'
    df.loc[df['MSA'].str.contains('Weirton'),'state_key']='WV-OH'
    return(df)

In [4]:
'''
Function
--------
get_file_name

Get the appropriate file name giving year and table code

'''
def get_file_name(year, table_code):
    if year == 2006:
        mid = 'EST'
    else:
        mid = '1YR'
    return('ACS_'+str(year)[2:]+"_%s_" %mid + table_code)     

In [5]:
'''
Function
--------
convert_to_int

This function takes in a dataframe and list of vars to convert to int
'''
def convert_to_int(df, int_vars):
    df[int_vars] = df[int_vars].astype(int)
    return(df)
'''
Function
----------
create_proportions

This function will take in a list of variables and a single total variable
It then creates proportions by dividing each of the variables in the list by the total
to create a proportion
'''
def create_proportions(df,num_list, total_var):
    df.loc[:, num_list] = df[num_list].apply(lambda x: x / df[total_var])
    del df[total_var]
    return(df)

In [6]:
#####################
# Employment Data
#####################
emp_all = pd.DataFrame()
for year in range(2006, 2017):
    f = get_file_name(2006, 'S2301')
    employ = pd.read_csv("data/employ/%s.csv" %f, encoding='Latin-1')
    
    # Grab Unemployment
    un = [v for v in employ.columns if "HC04" in v and "EST" in v]
    employ = slim_df(employ, un)
    
    employ = employ.loc[:, ["MSA", "city_key", "state_key", 
                          "HC04_EST_VC01", "HC04_EST_VC03",
                         'HC04_EST_VC24']]
    employ['year'] = year
    emp_all = append_df(employ, emp_all) 

# Process Final DataFrame
emp_all = emp_all.sort_values(['city_key', 'state_key', 'year'])
emp_all = match_crime(emp_all)
del emp_all['MSA']
emp_all = emp_all.rename(index=str,
                        columns={'HC04_EST_VC01': 'unemp_16_ovr',
                                'HC04_EST_VC03': 'unemp_16_19',
                                'HC04_EST_VC24': 'unemp_female'})
emp_all.head()

,city_key,state_key,unemp_16_ovr,unemp_16_19,unemp_female,year
0,Abilene,TX,6.6,23.1,5.2,2006
367,Abilene,TX,6.6,23.1,5.2,2007
734,Abilene,TX,6.6,23.1,5.2,2008
1101,Abilene,TX,6.6,23.1,5.2,2009
1468,Abilene,TX,6.6,23.1,5.2,2010


In [7]:
############
# Age Data
############
age_all = pd.DataFrame()
for year in range(2006, 2017):
    f = get_file_name(year, 'S0101')
    age = pd.read_csv("data/age/%s.csv" %f, encoding='Latin-1')
    age = slim_df(age, [v for v in age.columns if "EST" in v])
    age = age.replace("(X)", np.nan)

    age = age.loc[:, ['MSA','city_key','state_key',
                      'HC01_EST_VC33','HC01_EST_VC34',
                      'HC01_EST_VC01', 'HC02_EST_VC01',
                      'HC03_EST_VC01', 'HC01_EST_VC06',
                      'HC01_EST_VC07', 'HC02_EST_VC07']]
    age['year'] = year
    age_all = append_df(age, age_all) 


# Process Final DataFrame
age_all = age_all.sort_values(['city_key', 'state_key', 'year'])
age_all = age_all.rename(index=str,
                         columns={'HC01_EST_VC33':'median_age',
                                'HC01_EST_VC34': 'sex_ratio',
                                'HC01_EST_VC01': 'total_pop',
                                'HC02_EST_VC01': 'male_pop',
                                'HC03_EST_VC01': 'female_pop',
                                'HC01_EST_VC06': 'pop_15_19',
                                'HC01_EST_VC07': 'pop_20_24',
                                'HC02_EST_VC07': 'male_pop_20_24'})

# Convert to Int and Get Proportions
age_all = convert_to_int(age_all, ['total_pop', 'male_pop', 'female_pop'])
age_all = create_proportions(age_all, ['male_pop', 'female_pop'], 'total_pop')
# Match Crime Data and then get rid of MSA
age_all = match_crime(age_all)
del age_all['MSA']
age_all.head()

,city_key,state_key,median_age,sex_ratio,male_pop,female_pop,pop_15_19,pop_20_24,male_pop_20_24,year
0,Abilene,TX,34.4,99.1,0.497717,0.502283,8.3,8.7,10.2,2006
367,Abilene,TX,34.9,99.1,0.497777,0.502223,9.5,7.7,8.6,2007
736,Abilene,TX,34.6,101.0,0.502381,0.497619,9.2,7.6,8.9,2008
1105,Abilene,TX,33.2,97.0,0.492269,0.507731,7.9,9.0,9.6,2009
1479,Abilene,TX,NaN,NaN,0.501355,0.498645,7.3,9.5,9.9,2010


In [8]:
###############
# Income Data
###############
inc_all = pd.DataFrame()
for year in range(2006, 2017):
    f = get_file_name(year, 'B19001F')
    inc = pd.read_csv("data/house_income/%s.csv" %f, encoding='Latin-1')
    # Keep only the estimates
    inc = slim_df(inc, [v for v in inc.columns if "HD01" in v])
    inc['year'] = year
    inc_all = append_df(inc, inc_all) 

# Proccess Final Data Frame
inc_all =  inc_all.rename(index=str,
                          columns={'HD01_VD01':'total',
                                  'HD01_VD02': 'inc_lt10',
                                  'HD01_VD03': 'inc_10_15',
                                  'HD01_VD04': 'inc_15_19',
                                  'HD01_VD05': 'inc_20_24',
                                  'HD01_VD06': 'inc_25_29',
                                  'HD01_VD07': 'inc_30_34',
                                  'HD01_VD08': 'inc_35_39',
                                  'HD01_VD09': 'inc_40_44',
                                  'HD01_VD10': 'inc_45_49',
                                  'HD01_VD11': 'inc_50_59',
                                  'HD01_VD12': 'inc_60_74',
                                  'HD01_VD13':'inc_75_99',
                                  'HD01_VD14':'inc_100_124',
                                  'HD01_VD15':'inc_125_149',
                                  'HD01_VD16':'inc_150_199',
                                  'HD01_VD17':'inc_gt_200'})

numeric_vars =  [v for v in inc_all.columns if "inc" in v]
inc_all = convert_to_int(inc_all, numeric_vars)
inc_all['total'] = inc_all['total'].astype(int)
# Get propotion of each imcome bracket by dividing by total
inc_all = create_proportions(inc_all, numeric_vars, "total")
# Match Crime data and Get rid of MSA
inc_all = match_crime(inc_all)
del inc_all['MSA']
inc_all.head()

,inc_lt10,inc_10_15,inc_15_19,inc_20_24,inc_25_29,inc_30_34,inc_35_39,inc_40_44,inc_45_49,inc_50_59,inc_60_74,inc_75_99,inc_100_124,inc_125_149,inc_150_199,inc_gt_200,city_key,state_key,year
0,0.066006,0.039355,0.191044,0.142857,0.034552,0.051751,0.094980,0.044159,0.004803,0.138829,0.060738,0.099628,0.004029,0.006508,0.018748,0.002014,Abilene,TX,2006
1,0.042883,0.102766,0.078153,0.191829,0.056077,0.063436,0.136514,0.011165,0.071048,0.108094,0.062928,0.046435,0.025121,0.003552,0.000000,0.000000,Albany,NY,2006
2,0.111215,0.064565,0.078066,0.069276,0.056352,0.080003,0.071902,0.061101,0.054006,0.094082,0.101140,0.101661,0.034098,0.007486,0.009591,0.005456,Albuquerque,NM,2006
3,0.150833,0.063611,0.084792,0.084514,0.105903,0.057778,0.043542,0.024583,0.080417,0.095417,0.075556,0.068958,0.042639,0.000000,0.021458,0.000000,Allentown,PA-NJ,2006
4,0.084392,0.086299,0.077275,0.135104,0.043213,0.134596,0.061642,0.051856,0.037239,0.120742,0.054270,0.070285,0.011185,0.022496,0.000000,0.009405,Amarillo,TX,2006


In [9]:
###############
# GINI INDEX
###############
gini_all = pd.DataFrame()
for year in range(2006, 2017):
    f = get_file_name(year, 'B19083')
    gini = pd.read_csv("data/gini/%s.csv" %f, encoding='Latin-1')
    # Don't need micro areas
    gini = slim_df(gini, ["HD01_VD01"])
    gini['year'] = year
    gini_all = append_df(gini, gini_all) 

# Clean Final Dataframes
gini_all = gini_all.rename(index=str,
                           columns={"HD01_VD01":"gini"})
gini_all['gini'] = gini_all['gini'].astype(float)
gini_all = match_crime(gini_all)
del gini_all['MSA']
gini_all.head()

,gini,city_key,state_key,year
0,0.443,Abilene,TX,2006
1,0.533,Aguadilla,PR,2006
2,0.445,Akron,OH,2006
3,0.481,Albany,GA,2006
4,0.405,Albany,NY,2006


In [10]:
#################
# Poverty Data
#################
pov_all = pd.DataFrame()
for year in range(2006, 2017):
    f = get_file_name(year, 'S1701')
    pov = pd.read_csv("data/poverty/%s.csv" %f, encoding='Latin-1')
    pov = slim_df(pov, ['HC03_EST_VC03', 'HC03_EST_VC05',
                       'HC03_EST_VC08', 'HC03_EST_VC09'])
    pov['year'] = year
    pov_all = append_df(pov, pov_all)
# Clean Final DataFrame
pov_all = pov_all.rename(index=str,
                        columns={'HC03_EST_VC03': 'under_18_pov',
                                'HC03_EST_VC05':'18_64_pov',
                                'HC03_EST_VC08':'male_pov',
                                'HC03_EST_VC09':'female_pov'})
pov_all = match_crime(pov_all)
del pov_all['MSA']
pov_all.head()

,under_18_pov,18_64_pov,male_pov,female_pov,city_key,state_key,year
0,20.4,14.9,15.5,16.1,Abilene,TX,2006
1,67.4,53.4,56.3,57.6,Aguadilla,PR,2006
2,15.7,12.5,10.7,14.6,Akron,OH,2006
3,31.0,20.7,20.7,24.7,Albany,GA,2006
4,13.2,8.6,8.7,10.9,Albany,NY,2006


In [11]:
#################################
# Head of Household Information
#################################
house_all = pd.DataFrame()
for year in range(2006, 2017):
    f = get_file_name(year, 'B09005')
    house = pd.read_csv("data/house_head/%s.csv" %f, encoding='Latin-1')
    house = slim_df(house, [v for v in house.columns if "HD01" in v])
    house = house.loc[:, ["MSA", "city_key", "state_key",
                         "HD01_VD01", "HD01_VD03", "HD01_VD05",
                         "HD01_VD06"]]
    house['year'] = year
    house_all = append_df(house, house_all) 

# Clean Entire DataFrame
house_all = house_all.rename(index=str,
                             columns={'HD01_VD01': 'total',
                                     'HD01_VD03': 'married_house',
                                     'HD01_VD05': 'female_house',
                                     'HD01_VD06': 'male_house'})

house_all = convert_to_int(house_all,
                       ['total', 'married_house', 'female_house', 'male_house'])
house_all = create_proportions(house_all, ['married_house', 'female_house', 'male_house'], 'total')
house_all = match_crime(house_all)
del house_all['MSA']
house_all.head()

,city_key,state_key,married_house,female_house,male_house,year
0,Abilene,TX,0.661882,0.253355,0.007038,2006
1,Aguadilla,PR,0.625417,0.324838,0.004308,2006
2,Akron,OH,0.686886,0.246946,0.008047,2006
3,Albany,GA,0.505709,0.429540,0.000000,2006
4,Albany,NY,0.668823,0.251285,0.013663,2006


In [12]:
#################
# Education Data
#################
edu_all = pd.DataFrame()
for year in range(2006, 2017):
    f = get_file_name(year, 'S1501')
    edu = pd.read_csv("data/education/%s.csv" %f, encoding='Latin-1')
    edu = slim_df(edu, ['HC01_EST_VC02', 'HC01_EST_VC03', 'HC01_EST_VC07',
                       'HC01_EST_VC08', 'HC01_EST_VC09'])
    edu['year'] = year
    edu_all = append_df(edu, edu_all)

edu_all = edu_all.rename(index=str,
                        columns={'HC01_EST_VC02': 'no_hs_18_24',
                                'HC01_EST_VC03': 'hs_18_24',
                                'HC01_EST_VC07': 'no_9th_25_ovr',
                                'HC01_EST_VC08':'no_hs_25_ovr',
                                'HC01_EST_VC09': 'hs_25_ovr'})

# Trim Final Dataframe
edu_all = match_crime(edu_all)
del edu_all['MSA']
edu_all.head()

,no_hs_18_24,hs_18_24,no_9th_25_ovr,no_hs_25_ovr,hs_25_ovr,city_key,state_key,year
0,10.8,30.2,7.5,14.5,28.6,Abilene,TX,2006
1,23.4,30.6,32.7,11.7,25.0,Aguadilla,PR,2006
2,11.6,34.1,2.4,8.9,34.4,Akron,OH,2006
3,29.1,22.1,7.0,12.2,31.5,Albany,GA,2006
4,11.9,29.7,3.2,6.9,30.5,Albany,NY,2006


In [13]:
############
# Race Data
############
race_all = pd.DataFrame()
for year in range(2006, 2017):
    f = get_file_name(year, 'B02001')
    race = pd.read_csv("data/race/%s.csv" %f, encoding='Latin-1')
    race = slim_df(race, [v for v in race.columns if "HD01" in v])
    race = race.loc[:, ['MSA', 'city_key', 'state_key',
                       'HD01_VD01','HD01_VD02',
                       'HD01_VD03', 'HD01_VD05']]
    
    race['year'] = year
    race_all = append_df(race, race_all) 

# Proccess Final Data Frame
race_all =  race_all.rename(index=str,
                            columns={'HD01_VD01':'total',
                                    'HD01_VD02': 'white',
                                    'HD01_VD03': 'black',
                                    'HD01_VD05': 'asian'})

race_all = convert_to_int(race_all, ['total', 'white','black','asian'] )
race_all = create_proportions(race_all, ['white', 'black', 'asian'], 'total')
# Match Crime Data
race_all = match_crime(race_all)
race_all.head()

,MSA,city_key,state_key,white,black,asian,year
0,"Abilene, TX",Abilene,TX,0.741838,0.068295,0.014292,2006
1,"Aguadilla-Isabela-San Sebastián, PR",Aguadilla,PR,0.896527,0.019961,0.000758,2006
2,"Akron, OH",Akron,OH,0.844866,0.116880,0.017715,2006
3,"Albany, GA",Albany,GA,0.485957,0.494136,0.006355,2006
4,"Albany-Schenectady-Troy, NY",Albany,NY,0.867237,0.070216,0.030761,2006


In [14]:
# Bring Everything Together
census_df = race_all.copy()

merge_df = lambda df: census_df.merge(df,
                                     how='outer',
                                     on=['city_key','state_key','year'],
                                     indicator=True)

str_list = ['Employment', 'Age', 'Income', 'Head of House',
            'Education', 'Gini', 'Poverty']
df_list = [emp_all, age_all, inc_all, house_all, edu_all, gini_all, pov_all]
for i, df in enumerate(df_list):
    census_df = merge_df(df)
    print("%s Merge Stats" %str_list[i])
    print(census_df['_merge'].value_counts())
    del census_df['_merge']
    
'''

Household had ones where it looks like there may be mismatches but code below checked it

Code to check merges
names = census_df.loc[census_df._merge != "both", ['city_key', 'state_key', '_merge']]
names = names.sort_values['city_key', 'state_key']
names = names.drop_duplicates()
print(names.shape[0])
names.head(50)
'''

Employment Merge Stats
both          3967
left_only      151
right_only      70
Name: _merge, dtype: int64
Age Merge Stats
both          4148
left_only       40
right_only       6
Name: _merge, dtype: int64
Income Merge Stats
left_only     3161
both          1033
right_only       0
Name: _merge, dtype: int64
Head of House Merge Stats
both          4153
left_only       41
right_only       0
Name: _merge, dtype: int64
Education Merge Stats
both          4154
left_only       40
right_only       0
Name: _merge, dtype: int64
Gini Merge Stats
both          4154
left_only       40
right_only       0
Name: _merge, dtype: int64
Poverty Merge Stats
both          4154
left_only       40
right_only       0
Name: _merge, dtype: int64


'\n\nHousehold had ones where it looks like there may be mismatches but code below checked it\n\nCode to check merges\nnames = census_df.loc[census_df._merge != "both", [\'city_key\', \'state_key\', \'_merge\']]\nnames = names.drop_duplicates()\nprint(names.shape[0])\nnames.head(50)\n'

In [15]:
float_cols = census_df.columns.difference(["MSA", "city_key", "state_key","year"])
census_df[float_cols] = census_df[float_cols].astype(float)
census_df.head()

,MSA,city_key,state_key,white,black,asian,year,unemp_16_ovr,unemp_16_19,unemp_female,...,no_hs_18_24,hs_18_24,no_9th_25_ovr,no_hs_25_ovr,hs_25_ovr,gini,under_18_pov,18_64_pov,male_pov,female_pov
0,"Abilene, TX",Abilene,TX,0.741838,0.068295,0.014292,2006,6.6,23.1,5.2,...,10.8,30.2,7.5,14.5,28.6,0.443,20.4,14.9,15.5,16.1
1,"Aguadilla-Isabela-San Sebastián, PR",Aguadilla,PR,0.896527,0.019961,0.000758,2006,20.6,56.8,19.1,...,23.4,30.6,32.7,11.7,25.0,0.533,67.4,53.4,56.3,57.6
2,"Akron, OH",Akron,OH,0.844866,0.116880,0.017715,2006,6.3,23.0,4.8,...,11.6,34.1,2.4,8.9,34.4,0.445,15.7,12.5,10.7,14.6
3,"Albany, GA",Albany,GA,0.485957,0.494136,0.006355,2006,10.0,31.0,10.1,...,29.1,22.1,7.0,12.2,31.5,0.481,31.0,20.7,20.7,24.7
4,"Albany-Schenectady-Troy, NY",Albany,NY,0.867237,0.070216,0.030761,2006,5.4,16.8,4.4,...,11.9,29.7,3.2,6.9,30.5,0.405,13.2,8.6,8.7,10.9


In [16]:
#####################
# Bring in BEA Data
#####################
bea_gdp = pd.read_csv("data/BEA_real_GDP_pc.csv",skiprows=[0,1,2], header=1)
del bea_gdp['Fips']
bea_gdp= bea_gdp.iloc[1:, :].rename(index=str, columns={"Area": 'MSA'})
bea_gdp = pd.melt(bea_gdp, id_vars=["MSA"], var_name='year', value_name='real_pc_gdp')
bea_gdp = bea_gdp.loc[bea_gdp.MSA.notnull(), :]
bea_gdp['year'] = bea_gdp['year'].astype(int)
bea_gdp = bea_gdp.loc[bea_gdp.year >= 2006, :]

# Get rid of MSA in paranthesis
bea_gdp['MSA'] = bea_gdp['MSA'].str.replace(r"\(.*\)","")
bea_gdp = split_MSA(bea_gdp)
del bea_gdp['MSA']

In [17]:
census_df = merge_df(bea_gdp)
print("Bea Merge Stats")
print(census_df['_merge'].value_counts())
#del census_df['_merge']
census_df = census_df.sort_values(["MSA"])
# Look at left right combinations

Bea Merge Stats
both          3997
right_only     205
left_only      197
Name: _merge, dtype: int64


In [18]:
# Check to make sure that there were no typos
names = census_df.loc[census_df._merge != "both", ['city_key', 'state_key', '_merge']]
names = names.drop_duplicates()
print(names.shape[0])
del census_df["_merge"]

51


In [19]:
census_df.to_json("output/census_df.json")
census_df.head()

,MSA,city_key,state_key,white,black,asian,year,unemp_16_ovr,unemp_16_19,unemp_female,...,hs_18_24,no_9th_25_ovr,no_hs_25_ovr,hs_25_ovr,gini,under_18_pov,18_64_pov,male_pov,female_pov,real_pc_gdp
0,"Abilene, TX",Abilene,TX,0.741838,0.068295,0.014292,2006,6.6,23.1,5.2,...,30.2,7.5,14.5,28.6,0.4430,20.4,14.9,15.5,16.1,33978.0
2208,"Abilene, TX",Abilene,TX,0.832574,0.079190,0.015352,2012,6.6,23.1,5.2,...,27.3,105670.0,5.9,10.6,0.4851,26.1,20.4,NaN,19.4,35406.0
1835,"Abilene, TX",Abilene,TX,0.840689,0.085917,0.016213,2011,6.6,23.1,5.2,...,32.9,106134.0,6.4,9.3,0.4344,16.6,18.0,NaN,14.8,33964.0
2963,"Abilene, TX",Abilene,TX,0.782534,0.073649,0.015273,2014,6.6,23.1,5.2,...,29.9,106094.0,5.2,8.7,0.4626,20.6,16.0,NaN,13.7,39776.0
1466,"Abilene, TX",Abilene,TX,0.843283,0.074948,0.010210,2010,6.6,23.1,5.2,...,27.8,104387.0,6.8,10.8,0.4590,22.0,17.8,NaN,15.7,34004.0
